# Add fallbacks (添加回调)
在LLM应用程序中存在许多可能的故障点，无论是LLM API的问题、模型输出不佳、其他集成的问题等。fallbacks可以帮助您优雅地处理和隔离这些问题。

关键是，fallbacks不仅可以应用于LLM级别，还可以应用于整个可运行级别。


### Handling LLM API Errors

这可能是回退的最常见用例。对LLM API的请求可能因各种原因而失败 - API可能宕机，您可能已达到速率限制，任何数量的事情。因此，使用回退可以帮助防范这些类型的问题。

**重要提示**：默认情况下，许多LLM包装器会捕获错误并重试。在使用回退时，您很可能需要将其关闭。否则第一个包装器将继续重试而不会失败。

In [22]:
from langchain.chat_models import ChatOpenAI,ChatAnthropic

首先，让我们模拟一下如果我们从OpenAI收到了RateLimitError会发生什么。

In [23]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

In [ ]:
# Note that we set max_retries = 0 to avoid retrying on RateLimits, etc
openai_llm = ChatOpenAI(
  max_retries=0)
anthropic_llm = ChatAnthropic(
    )
llm = openai_llm.with_fallbacks([anthropic_llm])

In [17]:
# 让我们先使用OpenAI LLm，以显示我们遇到了一个错误。
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

Hit error


我们可以像llm一样使用“LLM with Fallbacks”。

In [19]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)

chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError as R:
        print(R)
        print("Hit error")

rate limit
Hit error
